In [39]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from dateutil import parser
import io
import base64
from IPython.display import HTML
from imblearn.under_sampling import RandomUnderSampler

In [40]:
chunk_num = pd.read_csv("./train_numeric.csv",     chunksize = 100)
chunk_dat = pd.read_csv("./train_date.csv"   ,     chunksize = 100)
chunk_cat = pd.read_csv("./train_categorical.csv", chunksize = 100)

sample_num = 20000
df_num = chunk_num.get_chunk(sample_num)
df_dat = chunk_dat.get_chunk(sample_num)
df_cat = chunk_cat.get_chunk(sample_num)

C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel\zmqshell.py:533: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,57,59,60,62,63,65,66,68,69,71,72,74,75,77,78,80,81,83,84,86,87,89,90,92,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,324,335,337,340,342,344,346,348,351,354,357,360,363,366,369,372,375,378,381,384,387,390,393,396,399,402,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,

In [41]:
df_num.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [42]:
df_cat.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# 全部 NaN な列をdropして、Nanを大きな値で置換。
print("before" + str(df_num.shape))
df_num.dropna(how='all', axis=1, inplace=True)
df_num.fillna(999, inplace=True)
print("after" + str(df_num.shape))

before(20000, 970)
after(20000, 970)


In [44]:
# 全部 NaN な列をdropして、Nanを大きな値で置換。
print("before" + str(df_cat.shape))
df_cat.dropna(how='all', axis=1, inplace=True)
df_cat.fillna(999, inplace=True)
print("after" + str(df_cat.shape))

before(20000, 2141)
after(20000, 848)


In [45]:
# カテゴリ変数のパース(仮実装で、値に変換するだけ)
# 最終的にはビットマップっぽいので、エンコーディングする。
from parse import *
def cat_data_to_val(x):
    if type(x) is str:
        r = parse("T{}", x)
        return float(r[0])
    else:
        return x


In [46]:

def is_bit_on(s, pos):
    if type(s) is str:
        r = parse("T{}", s)
        val = (int(r[0]))
        mask = 1 << pos
        if val & mask:
            return 1
        else:
            return 0
    else:
        return 0

In [47]:
def decode_categorical_data(df):
    for column_name, item in df.iteritems():
        if column_name != 'Id':
            vallist = item.unique()
            bitlist = []
            for c in vallist:
                if type(c) is str:
                    r = parse("T{}", c)
                    val = int(r[0])
                    if val < 0:
                        val = val + 4294967296
                    bitpos = int(np.log2(val))
                    bitlist.append(bitpos)

            for bit in bitlist:
                df[column_name + "_bit_" + str(bit)] = df[column_name].apply(lambda x:is_bit_on(x, bit))
            df.drop(column_name, axis=1, inplace=True)
            print('.')

    

In [48]:
decode_categorical_data(df_cat)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [49]:
df_cat.columns

Index(['Id', 'L0_S1_F25_bit_0', 'L0_S1_F27_bit_3', 'L0_S1_F29_bit_0',
       'L0_S1_F31_bit_3', 'L0_S2_F33_bit_0', 'L0_S2_F35_bit_5',
       'L0_S2_F37_bit_0', 'L0_S2_F39_bit_5', 'L0_S2_F41_bit_0',
       ...
       'L3_S47_F4146_bit_0', 'L3_S47_F4151_bit_0', 'L3_S47_F4156_bit_0',
       'L3_S47_F4161_bit_0', 'L3_S47_F4166_bit_0', 'L3_S47_F4171_bit_0',
       'L3_S47_F4176_bit_0', 'L3_S47_F4181_bit_0', 'L3_S47_F4186_bit_0',
       'L3_S47_F4191_bit_0'],
      dtype='object', length=985)

In [51]:
df =  pd.merge(df_cat, df_num, on=['Id'], how = 'left')

In [52]:
print(df[df.isnull().any(axis=1)])

Empty DataFrame
Columns: [Id, L0_S1_F25_bit_0, L0_S1_F27_bit_3, L0_S1_F29_bit_0, L0_S1_F31_bit_3, L0_S2_F33_bit_0, L0_S2_F35_bit_5, L0_S2_F37_bit_0, L0_S2_F39_bit_5, L0_S2_F41_bit_0, L0_S2_F43_bit_5, L0_S2_F45_bit_0, L0_S2_F47_bit_5, L0_S2_F49_bit_0, L0_S2_F51_bit_5, L0_S2_F53_bit_0, L0_S2_F55_bit_5, L0_S2_F57_bit_0, L0_S2_F59_bit_5, L0_S2_F61_bit_0, L0_S2_F63_bit_5, L0_S2_F65_bit_0, L0_S2_F67_bit_5, L0_S4_F105_bit_0, L0_S4_F107_bit_0, L0_S4_F108_bit_6, L0_S9_F151_bit_0, L0_S9_F154_bit_3, L0_S9_F154_bit_2, L0_S9_F154_bit_15, L0_S9_F154_bit_9, L0_S9_F156_bit_0, L0_S9_F159_bit_3, L0_S9_F159_bit_2, L0_S9_F159_bit_15, L0_S9_F159_bit_9, L0_S9_F161_bit_0, L0_S9_F164_bit_3, L0_S9_F164_bit_2, L0_S9_F164_bit_15, L0_S9_F164_bit_9, L0_S9_F166_bit_0, L0_S9_F169_bit_3, L0_S9_F169_bit_2, L0_S9_F169_bit_15, L0_S9_F169_bit_9, L0_S9_F171_bit_0, L0_S9_F174_bit_3, L0_S9_F174_bit_2, L0_S9_F174_bit_15, L0_S9_F174_bit_9, L0_S9_F176_bit_0, L0_S9_F179_bit_3, L0_S9_F179_bit_2, L0_S9_F179_bit_15, L0_S9_F179_bit

In [53]:
import xgboost as xgb


def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

 

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    rf = RandomForestClassifier(max_depth=5,n_estimators=10,class_weight = cl_weight)
    rf.fit(X_train, y_train)
    return rf





In [55]:
df_train, df_test = train_test_split(df_num)

# アンダーサンプリングする
df_train_ok = df_train[df_train['Response'] == 0]
df_train_ng = df_train[df_train['Response'] == 1]

df_train_ok_smp = df_train_ok.sample(frac=0.05, replace=True)



In [61]:

print("ok sample = " + str(df_train_ok_smp.shape))
print("ng sample = " + str(df_train_ng.shape))
df_train = pd.concat([df_train_ok_smp, df_train_ng])
print("train sample = " + str(df_train.shape))



ok sample = (746, 970)
ng sample = (90, 970)
train sample = (836, 970)


In [62]:
rf = train_with_r_forest(df_train)
X_test, y_test = separate_X_y(df_test)
y_pred = rf.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [63]:

from sklearn.metrics import confusion_matrix
cf_mat = confusion_matrix(y_test, y_pred)
cf_mat

array([[4941,   37],
       [  19,    3]], dtype=int64)

In [64]:
def calc_mcc(cf_mat):
    tn, fp, fn, tp = cf_mat.ravel()
    print(tn, fp, fn, tp)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc

mcc = calc_mcc(cf_mat)
print(mcc)
    

4941 37 19 3
0.0957910814345


In [60]:
df_desc.loc['max', np.isnan(df_desc.loc['max', :])]

NameError: name 'df_desc' is not defined

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthew_scorer = make_scorer(matthews_corrcoef)
scores = cross_val_score(clf, X, y, cv=5, scoring=matthew_scorer)

In [ ]:
df_cat

In [ ]:
df_num